<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/CWS_G115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 6.4 MB 11.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 2.2 MB/s 
     |████████████████████████████████| 968 kB 4.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 138 k

In [2]:
clmns = [
    "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15","Open 16","Close 16","Open 17","Close 17","Open 18","Close 18","Open 19","Close 19","Open 20","Close 20","Open 21","Close 21","Open 22","Close 22","Open 23","Close 23","Open 24","Close 24","Open 25","Close 25","Open 26","Close 26","Open 27","Close 27","Open 28","Close 28","Open 29","Close 29","Open 30","Close 30",
    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1,1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(30, data.shape[0]):
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        sugg = "sell"
        if data[i][3] > data[i][0]:
            sugg = "buy"
        arr = np.array(gggrow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [3]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [4]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [5]:
folder_name = extract_data(25)
to_par(folder_name,1)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1484
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160  
161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200  
201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240  
241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 26

In [6]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))


opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               31232     
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 512)               262656    
                                                                 
 dense_3 (Dense)             (None, 256)               131328    
                                                                 
 dense_4 (Dense)             (None, 256)               65792     
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                                 
 dense_6 (Dense)             (None, 2)                 5

In [7]:
model.fit(xTrain,yTrain,epochs=30,batch_size=128,validation_data=(xTest,yTest))

Epoch 1/30
1370/1370 [==============================] - 33s 23ms/step - loss: 0.5091 - accuracy: 0.7385 - val_loss: 0.4014 - val_accuracy: 0.8206
Epoch 2/30
1370/1370 [==============================] - 31s 23ms/step - loss: 0.2812 - accuracy: 0.8871 - val_loss: 0.2529 - val_accuracy: 0.9058
Epoch 3/30
1370/1370 [==============================] - 32s 23ms/step - loss: 0.1673 - accuracy: 0.9416 - val_loss: 0.2159 - val_accuracy: 0.9252
Epoch 4/30
1370/1370 [==============================] - 32s 23ms/step - loss: 0.1261 - accuracy: 0.9581 - val_loss: 0.1866 - val_accuracy: 0.9400
Epoch 5/30
1370/1370 [==============================] - 31s 23ms/step - loss: 0.1048 - accuracy: 0.9660 - val_loss: 0.1848 - val_accuracy: 0.9413
Epoch 6/30
1370/1370 [==============================] - 32s 23ms/step - loss: 0.0893 - accuracy: 0.9709 - val_loss: 0.1952 - val_accuracy: 0.9446
Epoch 7/30
1370/1370 [==============================] - 32s 23ms/step - loss: 0.0762 - accuracy: 0.9741 - val_loss: 0.1840 -

In [8]:
model.save("CWS_G115_9531.h5")

In [12]:
yf.download("btc-usd",period="max",interval="1wk")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-15,465.864014,468.174011,384.532013,398.821014,398.821014,156903400
2014-09-22,399.100006,441.557007,374.332001,377.181000,377.181000,186772600
2014-09-29,376.928009,391.378998,289.295990,320.510010,320.510010,276657896
2014-10-06,320.389008,382.726013,302.559998,378.549011,378.549011,341152804
2014-10-13,377.920990,411.697998,368.897003,389.545990,389.545990,156902070
...,...,...,...,...,...,...
2022-04-04,46445.273438,47106.140625,42021.207031,42207.671875,42207.671875,188557001876
2022-04-11,42201.039062,42424.589844,39373.058594,39716.953125,39716.953125,174652159709
2022-04-18,39721.203125,42893.582031,38696.191406,39469.292969,39469.292969,184314843516


In [10]:
def process_for_prediction(data):
        i = -1
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten().reshape(-1,1)
        return arr
def make_prediction(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [13]:
make_prediction("btc-usd","max","1wk")

[*********************100%***********************]  1 of 1 completed


array([[0.9751985 , 0.03248462]], dtype=float32)

In [14]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 31):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])
        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)
        arr = np.array(gggrow).flatten()
        return arr
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_monthly,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [ ]:
make_prediction("s","x")